# Notebook For The Application

This notebook acts as a testing ground for the data handling, preprocessing, analysis and subsequent clustering.
You first get the data from the API, convert the json info to a pandas dataframe, and then perform the above-mentioned operations.

In [ ]:
import pandas as pd
import requests

In [67]:
# --- Step 1: Get the data from the API ---

# Data for the 'schools' table
school_response = requests.get('http://localhost:8000/schools/all?skip=0&limit=100')
# Data for the 'topics' table
topics_response = requests.get('http://localhost:8000/topics/all?skip=0&limit=100')
# Data for the 'users' table
users_response = requests.get('http://localhost:8000/users/all?skip=0&limit=100')
# Data for the 'quizzes' table
quiz_response = requests.get('http://localhost:8000/quizzes/all?skip=0&limit=100')
# Data for the 'questions' table
question_response = requests.get('http://localhost:8000/questions/all?skip=0&limit=1000')
# Data for the 'answers' table
answer_response = requests.get('http://localhost:8000/answers/all?skip=0&limit=1000')
# Data for the 'logs' table
logs_response = requests.get('http://localhost:8000/logs/all?skip=0&limit=1000')

In [68]:
# --- Step 2: Convert the JSON responses to DataFrames ---
school_df = pd.DataFrame(school_response.json())
topic_df = pd.DataFrame(topics_response.json())
user_df = pd.DataFrame(users_response.json())
quiz_df = pd.DataFrame(quiz_response.json())
question_df = pd.DataFrame(question_response.json())
answer_df = pd.DataFrame(answer_response.json())
log_df = pd.DataFrame(logs_response.json())

In [ ]:
# -- Preview the DataFrames to verify the data
school_df.head()

,name,province,area,type,id
0,Highland Park High School,Gauteng,urban,public,22
1,Riverside Academy,Western Cape,suburban,private,23
2,Sunrise Technical College,Limpopo,rural,public,24
3,Khulani Primary School,Eastern Cape,township,public,25
4,Greenwood Private School,KwaZulu-Natal,suburban,private,26


In [70]:
topic_df.head()

,name,details,id
0,Mechanics,"Forces, Motion, Energy, Momentum",23
1,Waves & Optics,"Sound, Light, Refraction, Diffraction",24
2,Electromagnetism,"Electric Fields, Circuits, Magnetism",25
3,Chemical Reactions,"Reaction Rates, Stoichiometry, Equilibrium",26
4,Organic Chemistry,"Hydrocarbons, Functional Groups, Polymers",27


In [71]:
student_df = user_df[user_df['type'] == 'student']
student_df['grade'] = student_df['grade'].astype(int)
student_df.head()

C:\Users\kizit\AppData\Local\Temp\ipykernel_22100\994225008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_df['grade'] = student_df['grade'].astype(int)


,email,name,surname,school_id,type,grade,id
2,peter.jones@example.com,Peter,Jones,24,student,10,11
3,mary.williams@example.com,Mary,Williams,25,student,9,12
4,david.brown@example.com,David,Brown,26,student,8,13
6,michael.taylor@example.com,Michael,Taylor,23,student,7,15
7,emily.wilson@example.com,Emily,Wilson,24,student,12,16


In [72]:
quiz_df.head()

,title,duration,grade,topic_id,school_id,id
0,Basic Mechanics,30,10,23,None,30
1,Waves and Sound,45,11,24,None,31
2,DC Circuits,40,10,25,None,32
3,Chemical Equilibrium,35,11,26,None,33
4,Hydrocarbons and Alkanes,60,12,27,None,34


In [73]:
log_df.sort_values(by=['question_id', 'user_id'])

,action,time,user_id,question_id,id
0,started,2025-05-10T10:00:00,11,1,1
1,paused,2025-05-10T10:01:00,11,1,2
2,resumed,2025-05-10T10:02:00,11,1,3
3,completed,2025-05-10T10:03:00,11,1,4
100,started,2025-05-10T12:05:00,12,1,101
...,...,...,...,...,...
499,completed,2025-05-10T20:23:00,15,25,500
596,started,2025-05-10T22:25:00,16,25,597
597,paused,2025-05-10T22:26:00,16,25,598
598,resumed,2025-05-10T22:27:00,16,25,599


In [90]:
answer_df.head()

,question_id,user_id,quiz_id,answer,id,marks,marksAchieved,question,correctAnswer,type
0,1,11,30,Newton,1,5,5,What is the SI unit of force?,Newton,text
1,1,12,30,Newton,26,5,5,What is the SI unit of force?,Newton,text
2,1,13,30,Newton,51,5,5,What is the SI unit of force?,Newton,text
3,1,14,30,Newton,76,5,5,What is the SI unit of force?,Newton,text
4,1,15,30,Newton,101,5,5,What is the SI unit of force?,Newton,text


In [75]:
# create a smaller question df with only id and level
small_q_df = question_df[['id', 'level']]
small_q_df.head()

# Merge answer_df with small_q_df on question_id and id to create a full dataframe
full_df = answer_df.merge(small_q_df, left_on='question_id', right_on='id', suffixes=('_answer', '_question'))

# drop the redundant id columns
full_df = full_df.drop(columns=['id_question', 'id_answer'], axis=1)

# reorder the columns to make more logical sense
full_df = full_df.iloc[:, [1, 2, 0, 6, 3, 7, 4, 5, 8, 9]]

full_df = full_df.sort_values(by=['user_id'])

# preview the full dataframe
full_df

,user_id,quiz_id,question_id,question,answer,correctAnswer,marks,marksAchieved,type,level
0,11,30,1,What is the SI unit of force?,Newton,Newton,5,5,text,low
6,11,30,2,Is velocity a scalar or vector quantity?,vector,vector,10,10,mc,medium
12,11,30,3,State Newton's first law of motion.,An object remains at rest...,An object remains at rest or in uniform motion...,15,15,text,high
18,11,30,4,What is the formula for kinetic energy?,KE = 1/2mv^2,$KE = \frac{1}{2}mv^2$,5,5,text,low
24,11,30,5,A car traveling at 60 km/h has a velocity of 6...,True,False,5,0,tf,low
...,...,...,...,...,...,...,...,...,...,...
23,16,30,4,What is the formula for kinetic energy?,KE = 1/2mv^2,$KE = \frac{1}{2}mv^2$,5,5,text,low
131,16,34,22,What is the simplest alkane?,E = mc^2,Methane,10,10,text,medium
137,16,34,23,What is the process of converting a liquid hyd...,Red,Distillation,15,5,text,high
143,16,34,24,What is the functional group of an alcohol?,273.15 K,-OH,5,5,text,low


In [76]:
# Convert 'time' to datetime objects for calculations
log_df['time'] = pd.to_datetime(log_df['time'])

# merge answers_df and log_df
merged_df = pd.merge(log_df, full_df, on=['user_id', 'question_id'], how='left')

merged_df

,action,time,user_id,question_id,id,quiz_id,question,answer,correctAnswer,marks,marksAchieved,type,level
0,started,2025-05-10 10:00:00,11,1,1,30,What is the SI unit of force?,Newton,Newton,5,5,text,low
1,paused,2025-05-10 10:01:00,11,1,2,30,What is the SI unit of force?,Newton,Newton,5,5,text,low
2,resumed,2025-05-10 10:02:00,11,1,3,30,What is the SI unit of force?,Newton,Newton,5,5,text,low
3,completed,2025-05-10 10:03:00,11,1,4,30,What is the SI unit of force?,Newton,Newton,5,5,text,low
4,started,2025-05-10 10:05:00,11,2,5,30,Is velocity a scalar or vector quantity?,vector,vector,10,10,mc,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,completed,2025-05-10 22:23:00,16,24,596,34,What is the functional group of an alcohol?,273.15 K,-OH,5,5,text,low
596,started,2025-05-10 22:25:00,16,25,597,34,Is ethanol a type of hydrocarbon?,1000g,False,5,10,tf,low
597,paused,2025-05-10 22:26:00,16,25,598,34,Is ethanol a type of hydrocarbon?,1000g,False,5,10,tf,low
598,resumed,2025-05-10 22:27:00,16,25,599,34,Is ethanol a type of hydrocarbon?,1000g,False,5,10,tf,low


# Feature Engineering

We must create meaningful numerical features from the merged data.  Since the goal is to determine the user's confidence level, we'll create two types of features: `behavioural` and `performance`.

## Behavioural Features from Log Data
- Time Taken per Question: Calculate the `total_elapsed_time` from for each question and subtract any time spent paused for `active_time`
- Number of Pauses: Count how many times a user paused on a question, which will assist in determining which questions they struggle more with

## Performance Features from Answers Data
- Success Rate: calculate the ratio of `marksAchieved` to `marks`.
- Average Question level: convert the `level` column to a numerical value and find the average


In [ ]:
# --- BEHAVIOURAL FEATURE ENGINEERING ---
# group by user and question to get start/end times
time_data = merged_df.groupby(['user_id', 'question_id']).agg(
    start_time=('time', 'min'),
    end_time=('time', 'max')
).reset_index()

# calculate total time elapsed
time_data['total_elapsed_time'] = (time_data['end_time'] - time_data['start_time']).dt.total_seconds()

# calculate time spent paused
# have to find pairs of 'paused' and 'resumed' events
paused_df = merged_df[merged_df['action'] == 'paused'].copy()
resumed_df = merged_df[merged_df['action'] == 'resumed'].copy()

# ensure we have a unique identifier for each pause/resume pair
paused_df['pair_id'] = paused_df.groupby(['user_id', 'question_id']).cumcount()
resumed_df['pair_id'] = resumed_df.groupby(['user_id', 'question_id']).cumcount()

# merge the paired dataframes to get start and end times for each pause
pause_pairs = pd.merge(
    paused_df[['user_id', 'question_id', 'time', 'pair_id']],
    resumed_df[['user_id', 'question_id', 'time', 'pair_id']],
    on=['user_id', 'question_id', 'pair_id'],
    how='inner',
    suffixes=('_start', '_end')
)

pause_pairs['pause_duration'] = (pause_pairs['time_end'] - pause_pairs['time_start']).dt.total_seconds()

# aggregate to get the total pause time per user and question
total_pause_time_df = pause_pairs.groupby(['user_id', 'question_id'])['pause_duration'].sum().reset_index(name='total_pause_time')

# Merge pause times back into main time data
time_data = pd.merge(time_data, total_pause_time_df, on=['user_id', 'question_id'], how='left').fillna(0)

# calculate active time
time_data['active_time'] = time_data['total_elapsed_time'] - time_data['total_pause_time']
time_data['active_time'] = time_data['active_time'].clip(lower=0)

# count number of pauses
pauses_count = merged_df[merged_df['action'] == 'paused'].groupby(['user_id', 'question_id']).size().reset_index(name='pause_count')
time_data = pd.merge(time_data, pauses_count, on=['user_id', 'question_id'], how='left').fillna(0)

time_data

,user_id,question_id,start_time,end_time,total_elapsed_time,total_pause_time,active_time,pause_count
0,11,1,2025-05-10 10:00:00,2025-05-10 10:03:00,180.0,60.0,120.0,1
1,11,2,2025-05-10 10:05:00,2025-05-10 10:08:00,180.0,60.0,120.0,1
2,11,3,2025-05-10 10:10:00,2025-05-10 10:13:00,180.0,60.0,120.0,1
3,11,4,2025-05-10 10:15:20,2025-05-10 10:18:20,180.0,60.0,120.0,1
4,11,5,2025-05-10 10:20:00,2025-05-10 10:23:00,180.0,60.0,120.0,1
...,...,...,...,...,...,...,...,...
145,16,21,2025-05-10 22:05:00,2025-05-10 22:08:00,180.0,60.0,120.0,1
146,16,22,2025-05-10 22:10:00,2025-05-10 22:13:00,180.0,60.0,120.0,1
147,16,23,2025-05-10 22:15:00,2025-05-10 22:18:00,180.0,60.0,120.0,1
148,16,24,2025-05-10 22:20:00,2025-05-10 22:23:00,180.0,60.0,120.0,1


In [93]:
# --- PERFORMANCE FEATURE ENGINEERING ---
# Add performance and level features from answers_df
answers_unique = full_df.drop_duplicates(subset=['user_id', 'question_id'])
user_features = pd.merge(time_data, answers_unique, on=['user_id', 'question_id'], how='left')
user_features['success_rate'] = user_features['marksAchieved'] / user_features['marks']
level_map = {'low': 1, 'medium': 2, 'high': 3}
user_features['level_numeric'] = user_features['level'].map(level_map)

print("Features per question (partial view):")
user_features


Features per question (partial view):


,user_id,question_id,start_time,end_time,total_elapsed_time,total_pause_time,active_time,pause_count,quiz_id,question,answer,correctAnswer,marks,marksAchieved,type,level,success_rate,level_numeric
0,11,1,2025-05-10 10:00:00,2025-05-10 10:03:00,180.0,60.0,120.0,1,30,What is the SI unit of force?,Newton,Newton,5,5,text,low,1.000000,1
1,11,2,2025-05-10 10:05:00,2025-05-10 10:08:00,180.0,60.0,120.0,1,30,Is velocity a scalar or vector quantity?,vector,vector,10,10,mc,medium,1.000000,2
2,11,3,2025-05-10 10:10:00,2025-05-10 10:13:00,180.0,60.0,120.0,1,30,State Newton's first law of motion.,An object remains at rest...,An object remains at rest or in uniform motion...,15,15,text,high,1.000000,3
3,11,4,2025-05-10 10:15:20,2025-05-10 10:18:20,180.0,60.0,120.0,1,30,What is the formula for kinetic energy?,KE = 1/2mv^2,$KE = \frac{1}{2}mv^2$,5,5,text,low,1.000000,1
4,11,5,2025-05-10 10:20:00,2025-05-10 10:23:00,180.0,60.0,120.0,1,30,A car traveling at 60 km/h has a velocity of 6...,True,False,5,0,tf,low,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,16,21,2025-05-10 22:05:00,2025-05-10 22:08:00,180.0,60.0,120.0,1,34,What is the general formula for an alkane?,False,$C_nH_{2n+2}$,5,15,mc,low,3.000000,1
146,16,22,2025-05-10 22:10:00,2025-05-10 22:13:00,180.0,60.0,120.0,1,34,What is the simplest alkane?,E = mc^2,Methane,10,10,text,medium,1.000000,2
147,16,23,2025-05-10 22:15:00,2025-05-10 22:18:00,180.0,60.0,120.0,1,34,What is the process of converting a liquid hyd...,Red,Distillation,15,5,text,high,0.333333,3
148,16,24,2025-05-10 22:20:00,2025-05-10 22:23:00,180.0,60.0,120.0,1,34,What is the functional group of an alcohol?,273.15 K,-OH,5,5,text,low,1.000000,1


In [96]:
# Aggregate all features to get a single row per user
final_df = user_features.groupby(['user_id', 'quiz_id']).agg(
    avg_total_elapsed_time=('total_elapsed_time', 'mean'),
    avg_active_time=('active_time', 'mean'),
    total_pauses=('pause_count', 'sum'),
    avg_success_rate=('success_rate', 'mean'),
    avg_level=('level_numeric', 'mean')
).reset_index()

print("Final df for clustering:")
final_df

Final df for clustering:


,user_id,quiz_id,avg_total_elapsed_time,avg_active_time,total_pauses,avg_success_rate,avg_level
0,11,30,180.0,120.0,5,0.800000,1.6
1,11,31,180.0,120.0,5,0.800000,1.6
2,11,32,180.0,120.0,5,1.000000,1.2
3,11,33,180.0,120.0,5,0.800000,1.6
4,11,34,180.0,120.0,5,1.466667,1.6
5,12,30,180.0,120.0,5,0.800000,1.6
6,12,31,180.0,120.0,5,0.800000,1.6
7,12,32,180.0,120.0,5,1.000000,1.2
8,12,33,180.0,120.0,5,0.800000,1.6
9,12,34,180.0,120.0,5,1.466667,1.6


In [97]:
# --- Scaling and K-Means Clustering
# select the features for the clustering, including both time metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

X = final_df[['avg_total_elapsed_time', 'avg_active_time', 'total_pauses', 'avg_success_rate', 'avg_level']]

# scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform K-Means Clustering (e.g with 3 clusters)
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
final_df['user_cluster'] = kmeans.fit_predict(X_scaled)

print("Final dataframe with user clusters:")
final_df

Final dataframe with user clusters:


,user_id,quiz_id,avg_total_elapsed_time,avg_active_time,total_pauses,avg_success_rate,avg_level,user_cluster
0,11,30,180.0,120.0,5,0.800000,1.6,0
1,11,31,180.0,120.0,5,0.800000,1.6,0
2,11,32,180.0,120.0,5,1.000000,1.2,1
3,11,33,180.0,120.0,5,0.800000,1.6,0
4,11,34,180.0,120.0,5,1.466667,1.6,2
5,12,30,180.0,120.0,5,0.800000,1.6,0
6,12,31,180.0,120.0,5,0.800000,1.6,0
7,12,32,180.0,120.0,5,1.000000,1.2,1
8,12,33,180.0,120.0,5,0.800000,1.6,0
9,12,34,180.0,120.0,5,1.466667,1.6,2


Now that we have the clusters for the users, we can start the interpretation of the results.  The values in the user_cluster column have no inherent meaning, so we have to assign some meaning.
We'll do this by examining the average feature values of all characteristics of the data points within each cluster. This is how:

1. Analyzing the cluster centroids: A centroid is the average position of all data points in a cluster, by looking at the centroid's values for each feature, we can create a profile for the cluster
2. Comparing the clusters: By comparing the profiles of different clusters, we can give them meaningful names, namely "low confidence", "medium confidence" and "high confidence".